In [2]:
import torch as t
a = t.arange(12).view(3,4)
print(a)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


In [3]:
a[t.tensor([1, 2]), t.tensor([0, 2])]

tensor([ 4, 10])

In [7]:
a[t.tensor([1,2])[None,:], t.tensor([0, 2])[:,None]] # 不相同形状的index索引，满足广播法则
                                                     # 获取索引为[1,0]、[2,0]、[1,2]、[2,2]的元素

tensor([[ 4,  8],
        [ 6, 10]])

In [12]:
t.tensor([1,2])[None,:], t.tensor([0, 2])[:,None]   #先广播法则扩展，再索引

(tensor([[1, 2]]),
 tensor([[0],
         [2]]))

In [13]:
print(a[[2,0]])

tensor([[ 8,  9, 10, 11],
        [ 0,  1,  2,  3]])


In [15]:
print(a[[1, 2, 0], [3, 2, 1]])

tensor([ 7, 10,  1])


In [17]:
b = t.tensor([[1, -3, 2], [2, 9, -1], [-8, 4, 1]])
print(b)
b[b > 0] *= 2
print(b)

tensor([[ 1, -3,  2],
        [ 2,  9, -1],
        [-8,  4,  1]])
tensor([[ 2, -3,  4],
        [ 4, 18, -1],
        [-8,  8,  2]])


In [14]:
# 随机生成一组形状为(5,3,4)、0~9数字组成的张量
a = t.randint(1, 10, (5,3,4))
# 获取输出的unique list和索引
output, inverse_indices = t.unique(a, return_inverse=True)
print(output)
print(inverse_indices)
# 通过整数数组索引 还原原始tensor
a_generate = output[inverse_indices]
a_generate.equal(a)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([[[1, 3, 3, 5],
         [5, 4, 5, 1],
         [4, 5, 2, 5]],

        [[6, 8, 0, 1],
         [3, 8, 1, 8],
         [1, 4, 6, 5]],

        [[3, 1, 4, 3],
         [5, 7, 5, 6],
         [7, 6, 1, 4]],

        [[3, 2, 5, 4],
         [6, 7, 8, 2],
         [4, 0, 8, 3]],

        [[2, 5, 4, 5],
         [4, 6, 3, 6],
         [4, 4, 8, 1]]])


True

In [23]:
x = t.randint(1, 10, (1,7))
print(x)
x[:,t.tensor([3,4,1,2])]  #  整数数组索引，输出的size等于索引的size，可以大于x本身的size

tensor([[4, 2, 1, 9, 3, 4, 7]])


tensor([[9, 3, 2, 1]])

# nn.embedding学习

In [11]:
import torch
import torch.nn as nn


# 输入
x = torch.LongTensor([[1, 2], [4, 2],[4, 1],[3, 2]])
print(x.shape)
print(x)

print("----------------------------------------------------------------------")
# 调用nn.Embedding函数
# 一共5个词，每个词的词向量维度设置为6维
embeddings = nn.Embedding(5, 6, padding_idx=4)
print(embeddings(x).shape)
print(embeddings(x))

torch.Size([4, 2])
tensor([[1, 2],
        [4, 2],
        [4, 1],
        [3, 2]])
----------------------------------------------------------------------
torch.Size([4, 2, 6])
tensor([[[-0.4663,  0.5425,  0.0533, -1.5261,  0.3110,  0.1871],
         [-1.0019, -1.4731,  0.5315, -0.0386,  2.1083,  0.2508]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-1.0019, -1.4731,  0.5315, -0.0386,  2.1083,  0.2508]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.4663,  0.5425,  0.0533, -1.5261,  0.3110,  0.1871]],

        [[ 2.3033, -0.0814, -0.1821, -1.6813, -1.2886, -0.6068],
         [-1.0019, -1.4731,  0.5315, -0.0386,  2.1083,  0.2508]]],
       grad_fn=<EmbeddingBackward>)


# BCELoss()学习

In [12]:
import torch
import torch.nn as nn

m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(8, requires_grad=True)
target = torch.empty(8).random_(2)
output = loss(m(input), target)
print(output)
print(output.item())    #item()取出对应位置元素的值

#only one element tensors can be converted to Python scalars

tensor(0.6702, grad_fn=<BinaryCrossEntropyBackward>)
0.6702415347099304


In [13]:
def validate_loss(output, target, weight=None, pos_weight=None):
    # 处理正负样本不均衡问题
    if pos_weight is None:
        label_size = output.size()[1]
        pos_weight = torch.ones(label_size)
    # 处理多标签不平衡问题
    if weight is None:
        label_size = output.size()[1]
        weight = torch.ones(label_size)

    val = 0
    
    #两层循环是为了处理多标签二分类问题
    """
    什么是多标签分类？比如说给一篇文章分配话题，它既可以是科技类又可以是教育类，科技和教育就是这篇文章的两个标签。
    又比如判断一幅图中包含什么，它可能既包含房子又包含马路。房子和马路就是这幅图对应的两个标签。
    将每一种标签，看作是二分类。一个输入样本对应于多个标签，每个标签对应一个二分类（是或不是）。

    """
    for li_x, li_y in zip(output, target):          
        #print(li_x)
        #print(li_y)
        for i, xy in enumerate(zip(li_x, li_y)):
            x, y = xy
            loss_val = pos_weight[i] * y * math.log(x, math.e) + (1 - y) * math.log(1 - x, math.e)
            val += weight[i] * loss_val
    return -val / (output.size()[0] * output.size(1))

import torch
import torch.nn.functional as F
import torch.nn as nn
import math

# 单标签二分类
m = nn.Sigmoid()
weight = torch.tensor([0.8])
loss_fct = nn.BCELoss(reduction="mean", weight=weight)
input_src = torch.Tensor([[0.8], [0.9], [0.3]])
target = torch.Tensor([[1], [1], [0]])
print(input_src.size())
print(target.size())
output = m(input_src)

#print(list(zip(output, target)))
loss = loss_fct(output, target)
print(loss.item())

# 验证计算
validate = validate_loss(output, target, weight)
print(validate.item())

torch.Size([3, 1])
torch.Size([3, 1])
0.4177626371383667
0.4177626371383667


In [14]:
# 单标签二分类
weight = torch.tensor([0.8])
input_src = torch.Tensor([[0.8], [0.9], [0.3]])
target = torch.Tensor([[1], [1], [0]])
print(input_src.size())
print(target.size())
output = torch.sigmoid(input_src)
loss = F.binary_cross_entropy(output, target, weight=weight, reduction='mean')
print(loss.item())

# 验证计算
validate = validate_loss(output, target, weight)
print(validate.item())

torch.Size([3, 1])
torch.Size([3, 1])
0.4177626371383667
0.4177626371383667
